# Load Data

In [10]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyodbc 
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

In [3]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=10.51.44.143;UID=mlmt;PWD=Mlmt@124;Database=sopirs_new;")
            
query = """
            SELECT *
            FROM BCN_HOUR_TTML_4G
            WHERE 
                DATETIME_ID >='2022060100'
                and TTML='Mien Trung'
        """

df = pd.read_sql_query(query, cnxn)
# df.to_csv("./play_data/kpi_4g_hour_20220601_20221012.csv")

In [124]:
df1 = df[["DATE", "HOUR", "TRAFFIC4G"]]
df1["DATE"] = pd.to_datetime(df['DATE'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')
df1["Time"] = df1["DATE"] + " " + df1["HOUR"] + ":00"

/tmp/ipykernel_55236/1446631874.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["DATE"] = pd.to_datetime(df['DATE'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')
/tmp/ipykernel_55236/1446631874.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Time"] = df1["DATE"] + " " + df1["HOUR"] + ":00"


# Build, train and evaluate model

In [125]:
timestamps, series = df1["Time"].values, df1["TRAFFIC4G"].values

In [126]:
# inject anomalies
from CellPAD.synthsiser import DropSynthesiser

syner = DropSynthesiser(raw_series=series, period_len=168)
syn_series, syn_labels = syner.syn_drop()

In [127]:
# detect drop
from CellPAD.controller import DropController

controller = DropController(timestamps=timestamps,
                            series=syn_series,
                            period_len=168,
                            feature_types=["Indexical", "Numerical"],
                            feature_time_grain=["Weekly"],
                            feature_operations=["Wma", "Ewma", "Mean", "Median"],
                            bootstrap_period_cnt=2,
                            to_remove_trend=True,
                            trend_remove_method="center_mean",
                            anomaly_filter_method="gauss",
                            anomaly_filter_coefficient=3.0)

In [128]:
controller.detect(predictor="RF")

/home/mlmt/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/mlmt/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/mlmt/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/mlmt/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/home/mlmt/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:396: F

In [129]:
#Evaluate results
from CellPAD.evaluator import evaluate

results = controller.get_results()
print(type(results))
print(results)
auc, prauc = evaluate(results["drop_scores"][2*168:], syn_labels[2*168:])

print("front_mean", "auc", auc, "prauc", prauc)

<class 'dict'>
{'drop_ratios': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000

In [130]:
model = controller.get_model()
test_array = np.array([[ 0.        ,  5.        ,  0.43346486,  0.43346486,  0.43346486,
         0.43346486,  0.43346486,  0.43346486,  0.43346486,  0.43346486,
         0.43346486,  0.43346486,  0.43346486,  0.43346486,  0.43346486,
         0.43346486,  0.43346486,  0.43346486]])
model.predict(test_array)

array([0.48229243])

# Predict

In [131]:
time1 = "2022101214"
time2 = "2022100514"
time3 = "2022092814"
time_array = [time1, time2, time3]

In [132]:
from datetime import datetime
import datetime

time_string = ""
for i in range(0,13):
    interval = 7*i
    time = (datetime.datetime.now() - datetime.timedelta(days=interval)).strftime("%Y%m%d%H")
    time_string = time_string + "\'" +time  + "\'" + ","
time_string = time_string[:-1]

query_predict = """
            SELECT *
            FROM BCN_HOUR_TTML_4G
            WHERE 
                DATETIME_ID in ({})
                and TTML='Mien Trung'
        """.format(time_string)
df_predict = pd.read_sql_query(query_predict, cnxn)
df_predict

,DATETIME_ID,INIT_ERAB_SR,CSFB_SR,TRAFFIC4G,ERAB_DR,TTML,DATE,HOUR,ERAB_SR_Status,CSFB_SR_Status,...,VoLTE_ERAB_SR,VoLTE_ERAB_DR,VoLTE_Pkt_Lost_DL_DR,VoLTE_Pkt_Lost_UL_DR,VoLTE_IntraHO_SR,VoLTE_SRVCC_HO_SR,VoLTE_SRVCC_HO_Pre_SR,VoLTE_SRVCC_HO_Pre_SUCC,VoLTE_SRVCC_HO_Pre_ATT,VoLTE_SRVCC_HO_SR_2
0,2022072115,99.91,99.92,58354.4,0.22,Mien Trung,21/07/2022,15,1.0,1.0,...,99.41,0.72,0.04,0.08,99.35,0.00,NaN,NaN,NaN,NaN
1,2022072815,99.92,99.92,54740.6,0.23,Mien Trung,28/07/2022,15,1.0,1.0,...,98.93,0.58,0.05,0.18,99.33,0.00,NaN,NaN,NaN,NaN
2,2022080415,99.77,99.92,55542.9,0.23,Mien Trung,04/08/2022,15,1.0,1.0,...,99.06,0.75,0.05,0.07,98.95,0.00,NaN,NaN,NaN,NaN
3,2022081115,99.89,99.91,56948.1,0.25,Mien Trung,11/08/2022,15,1.0,1.0,...,98.86,0.71,0.04,0.09,99.66,0.00,NaN,NaN,NaN,NaN
4,2022081815,99.91,101.72,55833.2,0.26,Mien Trung,18/08/2022,15,1.0,1.0,...,98.94,0.73,0.04,0.71,98.47,0.00,NaN,NaN,NaN,NaN
5,2022082515,99.86,99.92,55153.1,0.26,Mien Trung,25/08/2022,15,1.0,1.0,...,99.40,0.33,0.02,0.07,99.49,0.00,NaN,NaN,NaN,NaN
6,2022090115,99.79,99.92,57438.8,0.25,Mien Trung,01/09/2022,15,1.0,1.0,...,98.60,0.31,0.03,1.09,99.47,0.00,NaN,NaN,NaN,NaN
7,2022090815,99.85,99.92,52403.2,0.25,Mien Trung,08/09/2022,15,1.0,1.0,...,99.19,0.41,0.04,0.28,99.58,0.00,NaN,NaN,NaN,NaN
8,2022091515,99.91,99.92,54988.4,0.25,Mien Trung,15/09/2022,15,1.0,1.0,...,98.96,0.40,0.03,0.27,99.24,0.00,NaN,NaN,NaN,NaN
9,2022092215,99.70,99.91,58103.7,0.25,Mien Trung,22/09/2022,15,1.0,1.0,...,99.11,0.44,0.04,1.10,99.40,0.00,16.11,53.0,329.0,16.11


# -----------Test----------------

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

import pyodbc 
import numpy as np
import sys
np.set_printoptions(threshold=sys.maxsize)

cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=10.51.44.143;UID=mlmt;PWD=Mlmt@124;Database=sopirs_new;")
            
query = """
            SELECT *
            FROM BCN_HOUR_TTML_4G
            WHERE 
                DATETIME_ID >='2022060100'
                and TTML='Mien Trung'
        """

df = pd.read_sql_query(query, cnxn)

df1 = df
df1["DATE"] = pd.to_datetime(df['DATE'], format='%d/%m/%Y').dt.strftime('%Y/%m/%d')
df1["Time"] = df1["DATE"] + " " + df1["HOUR"] + ":00"
timestamps_traffic, series_traffic = df1["Time"].values, df1["TRAFFIC4G"].values
timestamps_ERAB, series_ERAB = df1["Time"].values, df1["INIT_ERAB_SR"].values

df_cellpad = pd.read_csv("./data/sd.csv")
timestamps_cellpad, series_cellpad = df_cellpad["Time"].values, df_cellpad["KPI"].values


In [2]:
from model import build_model, evaluate_regression_model as evaluate

model_cellpad, metrics_cellpad, features_cellpad = build_model(timestamps_cellpad, series_cellpad)
model_traffic, metrics_traffic, features_traffic = build_model(timestamps_traffic, series_traffic)
model_ERAB, metrics_ERAB, features_ERAB = build_model(timestamps_ERAB, series_ERAB)


In [3]:
metrics_cellpad

{'train_score': {'r2_score': 1.0,
  'mean_squared_error': 0.0,
  'Mean_absolute_error': 0.0},
 'test_score': {'r2_score': 0.7052192795876935,
  'mean_squared_error': 724775.8898809524,
  'Mean_absolute_error': 641.4940476190476}}

In [4]:
metrics_traffic

{'train_score': {'r2_score': 1.0,
  'mean_squared_error': 0.0,
  'Mean_absolute_error': 0.0},
 'test_score': {'r2_score': 0.9590904160224484,
  'mean_squared_error': 22346619.624332037,
  'Mean_absolute_error': 2748.287159533074}}

In [5]:
metrics_ERAB

{'train_score': {'r2_score': 0.9999409833919128,
  'mean_squared_error': 1.7821067821068278e-05,
  'Mean_absolute_error': 0.0003088023088026124},
 'test_score': {'r2_score': -0.2729524136796353,
  'mean_squared_error': 0.1392043305951868,
  'Mean_absolute_error': 0.17470817120622648}}

In [8]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};Server=10.51.44.143;UID=mlmt;PWD=Mlmt@124;Database=sopirs_new;")

query = """
            SELECT *
            FROM BCN_HOUR_CELL_4G
            WHERE 
                DATETIME_ID >='2022101300'
                and TTML='Mien Trung'
        """

df_cell = pd.read_sql_query(query, cnxn)

In [9]:
df.to_csv("tmp.csv")

,DATETIME_ID,INIT_ERAB_SR,CSFB_SR,TRAFFIC4G,ERAB_DR,TTML,DATE,HOUR,ERAB_SR_Status,CSFB_SR_Status,...,VoLTE_ERAB_DR,VoLTE_Pkt_Lost_DL_DR,VoLTE_Pkt_Lost_UL_DR,VoLTE_IntraHO_SR,VoLTE_SRVCC_HO_SR,VoLTE_SRVCC_HO_Pre_SR,VoLTE_SRVCC_HO_Pre_SUCC,VoLTE_SRVCC_HO_Pre_ATT,VoLTE_SRVCC_HO_SR_2,Time
0,2022060100,99.93,99.86,20271.0,0.08,Mien Trung,2022/06/01,00,1.0,1.0,...,0.00,0.01,0.04,93.33,100.00,NaN,NaN,NaN,NaN,2022/06/01 00:00
1,2022060101,99.95,100.04,12370.9,0.07,Mien Trung,2022/06/01,01,1.0,1.0,...,0.00,0.00,0.06,100.00,100.00,NaN,NaN,NaN,NaN,2022/06/01 01:00
2,2022060102,99.94,99.87,8802.8,0.07,Mien Trung,2022/06/01,02,1.0,1.0,...,3.23,0.00,0.02,100.00,0.00,NaN,NaN,NaN,NaN,2022/06/01 02:00
3,2022060103,99.94,99.93,7432.5,0.07,Mien Trung,2022/06/01,03,1.0,1.0,...,0.00,0.00,0.02,100.00,100.00,NaN,NaN,NaN,NaN,2022/06/01 03:00
4,2022060104,99.95,99.93,9067.6,0.07,Mien Trung,2022/06/01,04,1.0,1.0,...,0.00,0.01,0.02,100.00,100.00,NaN,NaN,NaN,NaN,2022/06/01 04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3244,2022101406,99.30,99.95,42268.3,0.16,Mien Trung,2022/10/14,06,1.0,1.0,...,1.00,0.02,0.07,99.37,100.00,95.00,38.0,40.0,95.00,2022/10/14 06:00
3245,2022101407,99.50,99.93,56244.2,0.20,Mien Trung,2022/10/14,07,1.0,1.0,...,0.65,0.04,0.07,99.40,97.87,71.21,47.0,66.0,69.70,2022/10/14 07:00
3246,2022101408,99.91,99.93,63312.1,0.23,Mien Trung,2022/10/14,08,1.0,1.0,...,0.43,0.03,0.41,99.37,100.00,15.85,42.0,265.0,15.85,2022/10/14 08:00
3247,2022101409,99.90,99.93,24191.0,0.25,Mien Trung,2022/10/14,09,1.0,1.0,...,0.40,0.04,0.13,99.33,93.55,18.45,31.0,168.0,17.26,2022/10/14 09:00
